In [129]:
import time
import pickle
from selenium import webdriver
from gsheet import authenticate
from selenium.webdriver.common.keys import Keys
import random

In [4]:
# Open Browser
browser = webdriver.Firefox()
# chrome = webdriver.Chrome()

In [131]:
# Constants
google = "https://google.com"
second_template_doc_url = "https://docs.google.com/document/d/120haq0-0NTiDSZRRmrP8ebmUggM6-jcJMUCTzUsIwoU/edit?usp=sharing"
outlook_office_url = "https://outlook.office.com/"
outlook_live_url = "https://outlook.live.com/"
fortest = "https://docs.google.com/spreadsheets/d/1UGhTjkslKEjoYA0mDe5l8GtS80hS39IsTe0YTE5s0a8/edit#gid=0"
gsheet_creds = "creds/s_a_creds.json"
tab_name = "Sheet2"
action = webdriver.ActionChains(browser)


def open_new_tab():
    browser.execute_script("window.open()")
    
    
def switch_to_tab(number):
    browser.switch_to.window(browser.window_handles[number])


# Functions

# Load Old Cookies (If exists)
def load_cookies(driver, location, url=None):
    cookies = pickle.load(open(location, "rb"))
    driver.delete_all_cookies()
    url = "https://www.google.com" if url is None else url
    driver.get(url)
    for cookie in cookies:
        driver.add_cookie(cookie)


# Credentials for used account
def get_email_creds(file_name):
    fl = open(file_name).readlines()
    email = fl[0]
    password = fl[1]
    return [email, password]


# Go to google docs url and copy the template
def grab_template(url):
    open_new_tab()
    switch_to_tab(1)
    browser.get(url)
    time.sleep(5)
    action.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).perform()
    time.sleep(1)
    action.key_down(Keys.CONTROL).send_keys("c").key_up(Keys.CONTROL).perform()
    switch_to_tab(0)

    
# Login to Normal Outlook Live Account
def login_to_outlook_live(email, password):
    browser.get("https://login.live.com")
    time.sleep(3)
    mail_id = browser.find_element_by_name("loginfmt")
    mail_id.send_keys(email)
    mail_id.send_keys(Keys.RETURN)
    time.sleep(2)
    passwd = browser.find_element_by_name("passwd")
    passwd.send_keys(password)
    passwd.submit()
    time.sleep(3)
    browser.get("https://outlook.live.com")


# Login to Professional Outlook Office account
def login_to_outlook_office(email, password):
    browser.get("https://outlook.office.com")
    time.sleep(3)
    mail_id = browser.find_element_by_name("loginfmt")
    mail_id.send_keys(email)
    mail_id.send_keys(Keys.RETURN)
    time.sleep(2)
    passwd = browser.find_element_by_name("passwd")
    passwd.send_keys(password)
    passwd.submit()
    time.sleep(3)
    browser.get("https://outlook.office.com")
    

# Connect To The Google Sheets API
def connect_to_gsheet(credentials, url, tab_name):
    client = authenticate(credentials)
    workbook = client.open_by_url(url)
    selected_tab = workbook.worksheet(tab_name)
    return selected_tab
    
    


In [132]:
# use outlook live account
# email_creds = get_email_creds("creds/outlook_neela.txt")
# login_to_outlook_live(email_creds[0], email_creds[1])
# time.sleep(2)
# pickle.dump(browser.get_cookies(), open("cookies/outlook_neela.txt", "wb"))

# Use Outlook Office Account
# try:
#     load_cookies(browser, "cookies/outlook_sarah.txt", url=outlook_office_url)
# except:
#     email_creds = get_email_creds("creds/outlook_sarah.txt")
#     login_to_outlook_office(email_creds[0], email_creds[1])
#     time.sleep(2)
#     pickle.dump(browser.get_cookies(), open("cookies/outlook_sarah.txt", "wb"))


selected_tab = connect_to_gsheet(gsheet_creds, fortest, tab_name)

In [133]:
# browser.get(outlook_live_url)
grab_template(second_template_doc_url)

In [135]:
# Select rows and send mail to them
print("Google Sheet Row Data:")
starting_row = int(input("Enter the Starting Row Number: "))
finishing_row = int(input("Enter the Finishing Row Number: "))
first_names = selected_tab.get(f"E{starting_row}:E{finishing_row}")
emails = selected_tab.get(f"H{starting_row}:H{finishing_row}")
total_rows = finishing_row - starting_row + 1
print(f"You have selected {total_rows} rows.\n")
# Loop
for i in range(total_rows):
    email = emails[i][0]
    first_name = first_names[i][0]
    print(f"Sending followup to {email}")
    try:
        search_bar = browser.find_element_by_xpath("//*[@aria-label='Search']")
        search_bar.click()
        search_bar.clear()
        search_bar.send_keys(email)
        search_bar.send_keys(Keys.ENTER)
        time.sleep(4)
        search_result_list = browser.find_element_by_xpath("//*[@aria-label='Message list No conversations selected']")
        search_result_list.send_keys(Keys.DOWN)
        time.sleep(2)
        content_pane = browser.find_element_by_xpath("//*[@aria-label='Content pane']")
        delivery_check = content_pane.text
        failing_texts = [
            "delivery has failed",
            "couldn't be delivered",
            "permanent fatal errors",
            "address not found",
            "your message wasn't delivered"
        ]
        is_failed = False
        for text in failing_texts:
            if text in delivery_check.lower():
                is_failed = True

        if not is_failed:
            reply = browser.find_element_by_name("Reply")
            reply.click()
        #     switch_to_tab(1)
        #     action.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).perform()
        #     time.sleep(1)
        #     action.key_down(Keys.CONTROL).send_keys("c").key_up(Keys.CONTROL).perform()
        #     time.sleep(1)
        #     switch_to_tab(0)
            msg_body = browser.find_element_by_xpath("//*[@aria-label='Message body']")
            msg_body.send_keys(f"Hi {first_name}-\n\n")
            time.sleep(3)
            msg_body.send_keys(Keys.CONTROL + "v")
            time.sleep(2)
            msg_body.send_keys(Keys.BACKSPACE)
            time.sleep(random.randint(10,30))
            msg_body.send_keys(Keys.CONTROL + Keys.ENTER)
            print(f"Successful: {email}")
        else:
            print(f"Failed Mail: {email}")
    except:
        print(f"Error Faced: {email}")
    time.sleep(2)
    if i == total_rows - 1:
        print(f"\n{total_rows} emails sent.")

Google Sheet Row Data:
Enter the Starting Row Number: 915
Enter the Finishing Row Number: 920
You have selected 6 rows.

Sending followup to naziursweet@gmail.com
Successful: naziursweet@gmail.com
Sending followup to mahmudur.rahman99@outlook.com
Successful: mahmudur.rahman99@outlook.com
Sending followup to sahebbazaar@gmail.com
Successful: sahebbazaar@gmail.com
Sending followup to slkdfja@lksjfkahhf.com
Failed Mail: slkdfja@lksjfkahhf.com
Sending followup to msdfkars@marsdecoder.com
Error Faced: msdfkars@marsdecoder.com
Sending followup to nesdfsdfela@marsdecodsfdfsder.com
Error Faced: nesdfsdfela@marsdecodsfdfsder.com

6 emails sent.
